In [1]:
import py2neo

airport_names = ["Dublin", "Paris","Toulouse", "Bordeaux", "London", "Malaga"]
airport_countries = ["Ireland","France","France","France","Britain","Spain"]

airport_dict = {}

for name, country in zip(airport_names, airport_countries):
    l = airport_dict.get(country, [])
    l.append(name)
    airport_dict[country] = l
    
print airport_dict
WEEKDAYS = ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]

print "CREATE", ",".join ( [ '({name}:Airport {{name:"{name}"}})'.format(name=a) for a in airport_names])

{'Britain': ['London'], 'Spain': ['Malaga'], 'Ireland': ['Dublin'], 'France': ['Paris', 'Toulouse', 'Bordeaux']}
CREATE (Dublin:Airport {name:"Dublin"}),(Paris:Airport {name:"Paris"}),(Toulouse:Airport {name:"Toulouse"}),(Bordeaux:Airport {name:"Bordeaux"}),(London:Airport {name:"London"}),(Malaga:Airport {name:"Malaga"})


In [2]:
from py2neo import authenticate, Graph, Relationship
g = Graph("http://localhost:7474/db/data/")


In [3]:
g.delete_all()
from py2neo import Node
try:
    g.schema.create_uniqueness_constraint("Airport", "name")
    g.schema.create_uniqueness_constraint("Weekday", "weekday")
except:
    pass

import itertools

for country, airports in airport_dict.iteritems():
    print country, "and its airports:", airports
    nodes = [Node("Airport", name=a) for a in airports]
    for node in nodes:
        g.create( node )
    
    if len(nodes) > 1:
        same_country = list(itertools.combinations ( nodes , 2 ))
        print same_country, "list"
        for a,b in same_country:
            g.create(Relationship(a, "Same_Country", b))
            
for day in WEEKDAYS:
    g.create ( Node ( "Weekday" , day) )

Britain and its airports: ['London']
Spain and its airports: ['Malaga']
Ireland and its airports: ['Dublin']
France and its airports: ['Paris', 'Toulouse', 'Bordeaux']
[((paris:Airport {name:"Paris"}), (toulouse:Airport {name:"Toulouse"})), ((paris:Airport {name:"Paris"}), (bordeaux:Airport {name:"Bordeaux"})), ((toulouse:Airport {name:"Toulouse"}), (bordeaux:Airport {name:"Bordeaux"}))] list


In [4]:
airports = list(g.find("Airport"))
weekdays = list(g.find("Weekday"))

In [5]:
import random
potential_flights = list ( itertools.permutations ( airports, 2) )
random.seed(117)

for i in range(16):
    
    cost = random.randint(20,100)
    to, dest = random.choice(potential_flights)
    weekday = random.choice(weekdays)
    flight_name = to['name'][:2]+'-'+dest['name'][:2]+'-'+str(random.randint(101,1000))
    print to, dest
    flight_node = Node("Flight", flight_name = flight_name, flight_cost = cost)
    out, landing = Relationship(to, "Flight_Out", flight_node), Relationship(flight_node, "Flight_In", dest)
    print out
    print landing
    g.create(flight_node )
    g.create(out)
    g.create(landing)
    print flight_node

(e7e7370:Airport {name:"London"}) (bordeaux:Airport {name:"Bordeaux"})
(e7e7370)-[:Flight_Out]->(dc04921)
(dc04921)-[:Flight_In]->(bordeaux)
(dc04921:Flight {flight_cost:39,flight_name:"Lo-Bo-243"})
(e7e7370:Airport {name:"London"}) (toulouse:Airport {name:"Toulouse"})
(e7e7370)-[:Flight_Out]->(c866df0)
(c866df0)-[:Flight_In]->(toulouse)
(c866df0:Flight {flight_cost:52,flight_name:"Lo-To-610"})
(toulouse:Airport {name:"Toulouse"}) (bordeaux:Airport {name:"Bordeaux"})
(toulouse)-[:Flight_Out]->(a01fc75)
(a01fc75)-[:Flight_In]->(bordeaux)
(a01fc75:Flight {flight_cost:48,flight_name:"To-Bo-978"})
(e7e7370:Airport {name:"London"}) (bordeaux:Airport {name:"Bordeaux"})
(e7e7370)-[:Flight_Out]->(d089a10)
(d089a10)-[:Flight_In]->(bordeaux)
(d089a10:Flight {flight_cost:55,flight_name:"Lo-Bo-481"})
(toulouse:Airport {name:"Toulouse"}) (baf9572:Airport {name:"Malaga"})
(toulouse)-[:Flight_Out]->(e01b307)
(e01b307)-[:Flight_In]->(baf9572)
(e01b307:Flight {flight_cost:20,flight_name:"To-Ma-634"})
(

In [6]:

flights_from_malaga = """
MATCH (malaga:Airport) - [out:Flight_Out] - (f:Flight) - [in:Flight_In] - (dest:Airport)
WHERE malaga.name='Malaga' 
RETURN  *
"""
from itertools import groupby

results = list(g.run(flights_from_malaga))

print "There are {0} results \n" .format ( str(len(results)) ) 

import operator

results_dict = {}

for key, group in groupby(results, lambda result: result['dest']['name']):
    key_dict = results_dict.get(key, [])
    key_dict.extend( group )
    results_dict[key] = key_dict
    
for key in results_dict.keys():
    print key, min ( results_dict[key] , key = operator.itemgetter(1) )['f']['flight_cost']

There are 4 results 

Paris 46
Toulouse 20


In [7]:
# Not sure why identifiers != name
for node in airports:
    print node

(e7e7370:Airport {name:"London"})
(baf9572:Airport {name:"Malaga"})
(f560c4b:Airport {name:"Dublin"})
(paris:Airport {name:"Paris"})
(toulouse:Airport {name:"Toulouse"})
(bordeaux:Airport {name:"Bordeaux"})
